In [1]:
import nbimporter
import codecs
#from RDFmanagement import *
from rdflib import Graph, Literal, BNode, Namespace, RDF,RDFS, URIRef
from spacy.tokens import Span
from RDFmanagement import *

Importing Jupyter notebook from RDFmanagement.ipynb


In [2]:
def stopwordsList(filename):
    #Cargar lista de stopwords
    stopwordslist = []
    stopfile= codecs.open(filename,"r",encoding="UTF-8")
    for line in stopfile:
        stop = line
        stop2 = stop.replace("\n","")
        stopwordslist.append(stop2)
    return stopwordslist

In [3]:
def replace_tildes(es_string):
    es_string1 = es_string.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u").replace("Á", "A").replace("É", "E").replace("Í", "I").replace("Ó", "O").replace("Ú", "U")
    return es_string1

In [4]:
def generateTriplesByDetectedLang(doc,graph, sentenceNode,lang_str,idtoken, idsentence, stoplist_lang, stoplist_punt):
    # RDF: Seteando propiedades y clases
    vutplSchema = Namespace('http://data.utpl.edu.ec/vutpl/vocabulary/')
    vutplResource = Namespace('http://data.utpl.edu.ec/vutpl/resource/')

    sentencetype = URIRef(vutplSchema+'Sentence')
    tokentype = URIRef(vutplSchema+'Token')
    roltype = URIRef(vutplSchema+'Rol')
    chunktype = URIRef(vutplSchema+'Chunk')
    dependencytype = URIRef(vutplSchema+'Dependency')
    postagtype = URIRef(vutplSchema+'PosTag')
    entitytype = URIRef(vutplSchema+'Entity')

    #Definir Clases
    grade = URIRef(vutplSchema+'Grade/')
    subject = URIRef(vutplSchema+'Subject/')
    plan = URIRef(vutplSchema+'TeachingPlan/')
    section = URIRef(vutplSchema+'Section/')
    sentence = URIRef(vutplResource+'Sentence/')
    tokenURI= URIRef(vutplResource+'Token/')
    rol = URIRef(vutplResource+'Rol/')
    chunk = URIRef(vutplResource+'Chunk/')
    dependency = URIRef(vutplResource+'Dependency/')
    postag = URIRef(vutplResource+'PosTag/')
    entity = URIRef(vutplResource+'Entity/')

    #Definir Propiedades
    hasSubject = URIRef(vutplSchema+'hasSubject')
    hasTeachingPlan = URIRef(vutplSchema+'hasTeachingPlan')
    hasSection = URIRef(vutplSchema+'hasSection')
    hasSentence = URIRef(vutplSchema+'hasSentence')
    sentenceId = URIRef(vutplSchema+'sentenceId')
    dataSentence = URIRef(vutplSchema+'dataSentence')
    hasToken= URIRef(vutplSchema+'hasToken')
    pnp = URIRef(vutplSchema+'pnp')
    tokenId = URIRef(vutplSchema+'tokenId')
    pos = URIRef(vutplSchema+'pos')
    data = URIRef(vutplSchema+'data')
    ner = URIRef(vutplSchema+'ner')
    nertype = URIRef(vutplSchema+'nertype')
    lemma = URIRef(vutplSchema+'lemma')
    hasRole = URIRef(vutplSchema+'hasRole')
    hasChunk = URIRef(vutplSchema+'hasChunk')
    
    for sent in doc.sents:
        for token in sent:
            if token.lemma_ not in stoplist_lang:
                if token.lemma_ not in stoplist_punt:
                    tokenNode = add_node(tokenURI+"Token" + str(idtoken) + "_Sentence" + str(idsentence))
                    add_graph(graph, tokenNode, RDF.type, tokentype)
                    add_graph(graph, sentenceNode, hasToken, URIRef(tokenNode))
                    add_graph(graph, tokenNode, tokenId, Literal(idtoken, lang=lang_str))
                    posTagNode = add_node(postag + token.pos_)
                    add_graph(graph, posTagNode, RDF.type, postagtype)
                    add_graph(graph, posTagNode, RDFS.label, Literal(token.pos_, lang=lang_str))
                    add_graph(graph, tokenNode, pos, posTagNode)
                    add_graph(graph, tokenNode, RDFS.label, Literal(token.pos_, lang=lang_str))
                    add_graph(graph, tokenNode, data, Literal(token.text, lang=lang_str))
                    add_graph(graph, tokenNode, lemma, Literal(token.lemma_, lang=lang_str))
                    #print(token.text, token.lemma_, token.pos_,token.is_stop)
                    idtoken += 1
        idtoken = 1
        idsentence += 1
    #for ent in doc.ents:
    #entityNode = add_node(entity + ent.label_)
    #add_graph(graph, entityNode, RDF.type, entitytype)
    #add_graph(graph, entityNode, RDFS.label, Literal(ent.label_, lang="en"))
    #add_graph(graph, sentenceNode, ner, Literal(ent.text, lang="en"))
    #add_graph(graph, sentenceNode, nertype, entityNode)
    #print(ent.text, ent.start_char, ent.end_char, ent.label_)
    return

In [5]:
#Guardar tripletas en archivo sql
def storeTriplesDBfile(grafo):
    lengg = len(grafo)
    iterador = 1
    f =codecs.open("/home/ela/Dropbox/ProyectoUTPL-NLP/NLPcode_Lady/jupyterproy/tripletas/tripletasSinStopwords.sql","wb",encoding="UTF-8")
    for tripleta in grafo:
        print ("Guardando " + str(iterador) + " de " + str(lengg))
        print (tripleta)
        sujeto1 = tripleta[0]
        predicado1 = tripleta[1]
        objeto1 = tripleta[2]
        f.write('INSERT into tripletas values  (%s, %s, %s)' % ('"' + sujeto1 + '"', '"' + predicado1 + '"', '"' + objeto1 + '"') + ';\n')
        iterador +=1
    return

In [6]:
# buscar en la lista de ners guardados cada oracion que ingrese, si la encuentra es uu NER que debe guardarse
def add_ner(doc, inicio, fin):
    ner_name = doc.vocab.strings[u'MISC']  # get hash value of entity label
    new_ent = Span(doc, inicio, fin, label=ner_name) # create a Span for the new entity
    doc.ents = list(doc.ents) + [new_ent]
    return doc.ents

In [7]:
def tokenizing(doc,stoplist_punt):
    solo_tokens=[]
    for sent in doc.sents:
        for token in sent:
            if token.text not in stoplist_punt:
                solo_tokens.append(token.text)      
    return solo_tokens

In [8]:
def lemmatizing(doc,stoplist_punt):
    solo_lemas = []
    for sent in doc.sents:
        for token in sent:
            #if token.lemma_ not in stoplist_lang:
            if token.lemma_ not in stoplist_punt:
                solo_lemas.append(token.lemma_)
                #print(token.text, token.lemma_, token.pos_,token.is_stop)         
    return solo_lemas